In [37]:
import os
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox

import numpy as np
import pandas as pd
import qgrid

import requests
from urllib.parse import urljoin
from jinja2 import Environment, FileSystemLoader
from IPython.display import display

In [20]:
base_url = 'http://127.0.0.1:8000'

In [21]:
session = requests.Session()

In [22]:
template_loader = FileSystemLoader(searchpath="./templates")
template_env = Environment(loader=template_loader)


In [23]:
all_topics_template = template_env.get_template('all_topics.html')
topic_template = template_env.get_template('show_topic.html')

In [24]:
def get_number_of_topics(session):
    r = session.get(urljoin(base_url, '/topics/number'))
    return r.json()['num_topics']

In [25]:
num_topics = get_number_of_topics(session)
num_topics

77

In [26]:
def get_topics(session, num_topics):
    payload = {'num_topics':num_topics}
    r = session.get(urljoin(base_url, '/topics/get-topics'), params=payload)
    return r.json()

In [27]:
topics = get_topics(session, num_topics)
topics_dict = {t['topic_num']: t['topic_words'] for t in topics}

In [28]:
top_words_in_topic = [(t['topic_num'], ', '.join(t['topic_words'][:10])) for t in topics]

In [29]:
rendered_all_topics = all_topics_template.render(topics=top_words_in_topic)

In [30]:
initial_topic_num = 0

In [31]:
topic_select_textbox = widgets.BoundedIntText(
                        value=initial_topic_num,
                        min=0,
                        max=num_topics,
                        step=1,
                        description='',
                        disabled=False,
                    )

In [42]:
topic_select_slider = widgets.IntSlider(
    value=initial_topic_num,
    min=0,
    max=num_topics,
    step=1,
    description='Topic:',
    disabled=False,
    orientation='horizontal',
    readout=False,
    readout_format='d',
)

In [43]:
topic_words_widget = widgets.HTML(
        value = topic_template.render(topic_num=initial_topic_num, topic=', '.join(topics_dict[initial_topic_num]))
    )

In [44]:
topic_num_link = widgets.jslink((topic_select_slider, 'value'), (topic_select_textbox, 'value'))

In [45]:
def handle_slider_change(change):
    topic_num = change.new
    
    if topic_num == num_topics:
        rendered_all_topics = all_topics_template.render(topics=top_words_in_topic)
        topic_words_widget.value = rendered_all_topics
    else:
        rendered_topic = topic_template.render(topic_num=topic_num, topic=', '.join(topics_dict[topic_num]))
        topic_words_widget.value = rendered_topic

topic_select_slider.observe(handle_slider_change, names='value')

In [46]:
top_box = HBox([topic_select_slider, topic_select_textbox])
VBox([top_box, topic_words_widget])

In [ ]:
payload = {'topic_num': 5, 'num_docs': 10}
results = session.`get(urljoin(base_url, '/documents/search-by-topic'), params=payload)

In [ ]:
results.json()

In [ ]:
payload = {'num_topics':77}
results = session.get(urljoin(base_url, '/topics/get-topics'), params=payload)
results.json()